In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_curve, auc
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model

In [4]:
traindata_day1 = pd.read_csv("./10_BOS_LeftLowerLimb/rawData_Day1_l.csv")
traindata_day1.head()

,Unnamed: 0,timestamp,leftAnkle_X,leftAnkle_Y,leftAnkle_Z,leftAnkle_Magnitude,Dysk_Score
0,0,1.429524e+09,-0.15686,-0.15686,9.88235,9.88484,0
1,1,1.429524e+09,-0,-0,9.72549,9.72549,0
2,2,1.429524e+09,-0.15686,-0.15686,9.88235,9.88484,0
3,3,1.429524e+09,-0.15686,-0.15686,9.88235,9.88484,0
4,4,1.429524e+09,-0.15686,-0,9.88235,9.8836,0


In [5]:
traindata_day4 = pd.read_csv("./10_BOS_LeftLowerLimb/rawData_Day4_l.csv")
traindata_day4.head()

,Unnamed: 0,timestamp,leftAnkle_X,leftAnkle_Y,leftAnkle_Z,leftAnkle_Magnitude,Dysk_Score
0,0,1.429747e+09,0.00000,9.72549,2.66667,10.08446,0
1,1,1.429747e+09,-0.15686,9.72549,2.66667,10.08568,0
2,2,1.429747e+09,-0.15686,9.56862,2.66666,9.93450,0
3,3,1.429747e+09,-0.31372,9.41177,2.35294,9.70650,0
4,4,1.429747e+09,0.00000,9.56863,2.66667,9.93327,0


Merge Patient 10_BOS data for days 1 and 4 to create a single dataframe that will be used for validation.

In [18]:
trainingdata = pd.concat([traindata_day1, traindata_day4])
print(len(trainingdata))
training_labels = trainingdata['Dysk_Score']

4717937


In [23]:
training_data = trainingdata[ ['leftAnkle_X', 'leftAnkle_Y','leftAnkle_Z'] ]

In [9]:
valdata_day1 = pd.read_csv("./17_BOS_LeftLowerLimb/rawData_Day1_l.csv")
valdata_day4 = pd.read_csv("./17_BOS_LeftLowerLimb/rawData_Day4_l.csv")
valdata_day1.head()

,Unnamed: 0,timestamp,leftAnkle_X,leftAnkle_Y,leftAnkle_Z,leftAnkle_Magnitude,Dysk_Score
0,0,1.444037e+09,0.15686,0,9.72549,9.72675,0
1,1,1.444037e+09,0.31372,0,9.56863,9.57377,0
2,2,1.444037e+09,0.15686,-0.15686,9.88235,9.88484,0
3,3,1.444037e+09,0.47059,-0.15686,10.0392,10.0515,0
4,4,1.444037e+09,0.15686,0.15686,9.56863,9.5712,0


Merge Patient 17_BOS data for days 1 and 4 to create a single dataframe that will be used for validation.

In [11]:
valdata = pd.concat([valdata_day1, valdata_day4])
print(len(valdata))
val_labels = valdata['Dysk_Score']
val_labels.head()

5307298


0    0
1    0
2    0
3    0
4    0
Name: Dysk_Score, dtype: int64

In [24]:
val_data = valdata[ ['leftAnkle_X', 'leftAnkle_Y','leftAnkle_Z'] ]

Set up hyperparameters

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Flatten, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D
from tensorflow.keras.optimizers import Adam, schedules, SGD

Very simple model to prototype
This is a sequential model, meaning the output of each layer is passed directly into the next one.
The first layer is Conv2D: This is a convolutional layer; it directly receives the network’s input, which is a sequence of raw accelerometer data. The input’s shape is provided in the input_shape argument. It’s set to (seq_length, 3, 1), where seq_length is the total number of accelerometer measurements that are passed in (128 by default). Each measurement is composed of three values, representing the x-, y-, and z-axes. 
The role of the convolutional layer is to take the raw data and extract some basic features that can be interpreted by subsequent layers. The arguments to the Conv2D() function determine how many features will be extracted.
The first argument determines how many filters the layer will have. During training, each filter learns to identify a particular feature in the raw data—for example, one filter might learn to identify the signs of an upward motion. For each filter, the layer outputs a feature map that shows where the feature it has learned occurs within the input.
The layer defined in the code has 8 filters, meaning that it will learn to recognize and output eight different types of high-level features from the input data. This is reflected in the output shape, (batch_size, 128, 3, 8), which has eight feature channels in its final dimension, one for each feature. The value in each channel indicates the degree to which a feature was present in that location of the input.
The convolutional layers slide a window across the data and decide whether a given feature is present in that window. The second argument to Conv2D() is where we provide the dimensions of this window. In our case, it’s (4, 3). This means that the features for which our filters are hunting span four consecutive accelerometer measurements and all three axes. Because the window spans four measurements, each filter analyzes a small snapshot of time, meaning it can generate features that represent a change in acceleration over time. 
When used for classification, the goal of a CNN is to transform a big, complex input tensor into a small, simple output. This is what the MaxPool2D layer does by congesting the output of the first convolutional layer into a concentrated, high-level representation of the most relevant features it contains. Even though the original input has 3 accelerometer axes for each measurement, a combination of Conv2D and MaxPool2D merges these together into a single value.
The Dropout layer randomly sets some of a tensor’s values to zero during training. In this case, by calling Dropout(0.1), we set 10% of the values to zero, entirely obliterating that data. Dropout is a regularization technique and process of improving machine learning models so that they are less likely to overfit their training data. By randomly removing some data between one layer and the next, we force the neural network to learn how to cope with unexpected noise and variation.
There can be more Conv2D and MaxPooling layers; let us strat with 2.
The Flatten layer is used to transform a multidimensional tensor into one with a single dimension. In this case, our (14, 1, 16) tensor is squished down into a single dimension with shape (224).
It’s then fed into a Dense layer with 16 neurons, which considersg all data at once, learning the meanings of various combinations of inputs. The output of this Dense layer will be a set of 16 values representing the content of the original input in a highly compressed form.

In [ ]:
# Define the base model
model = tf.keras.models.Sequential([tf.keras.layers.Conv2D(8, (4,3), activation=tf.nn.relu, padding="same", input_shape=(128,3,1), # output_shape=(batch, 128, 3, 8)),
                                    tf.keras.layers.MaxPooling2D(3,3),
                                    tf.keras.layers.Dropout(0.1),
                                    tf.keras.layers.Conv2D(16, (4,1), activation=tf.nn.relu, padding="same", #input_shape=(42,1,16), # output_shape=(batch, 128, 3, 8)),
                                    tf.keras.layers.MaxPooling2D(3,1),
                                    tf.keras.layers.Dropout(0.1),
                                    tf.keras.layers.Flatten(), # (batch, 224)
                                    tf.keras.layers.Dense(16, activation=tf.nn.relu), 
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])


In [ ]:
KERNEL_SIZE = 2
POOL_SIZE = 2
N_FILTERS = 24
BATCH_SIZE = 128
LEARNING_RATE = 0.00001
DENSE_UNITS = 50
LSTM_UNITS = 50
EPOCHS = 100
#TO DO: set the right shape
N_STEPS = X.shape[1]
#N_STEPS = X.shape[1]
#TO DO: set the right shape
N_FEATURES = (4,)
#N_FEATURES = X.shape[4]
#this line sets the class weight, which is then stated in the model below. 
#Need to check what the calculation of the right class weight is. 
#Should it be based on the share of a class in a training dataset?
class_weight = {0: 1., 1: 1/np.mean(training_labels)}

ACTIVATION_FUNC = ‘relu’
FINAL_ACTIVATION = ‘sigmoid’
LOSS_FUNC = ‘binary_crossentropy’
METRIC = ‘accuracy’
OPTIMIZER = Adam(learning_rate=LEARNING_RATE)

In [ ]:
model = Sequential()
# Add one or more convolutional layers
model.add(Conv2D(
    filters=N_FILTERS,
    padding=”same”,
    kernel_size=(2,2),
    activation=ACTIVATION_FUNC,
    input_shape=(N_STEPS, N_FEATURES, 1)),
)
model.add(MaxPooling2D(pool_size=2))
model.add(Flatten())
model.add(Dense(DENSE_UNITS, activation=ACTIVATION_FUNC))
model.add(Dense(1, activation=FINAL_ACTIVATION))
model.compile(
    optimizer=OPTIMIZER,
    loss=LOSS_FUNC,
    metrics=[tf.keras.metrics.AUC(), METRIC]
)

The model is a classifier with multiple classes. Thus, the fitting includes class weights. In most cases, classes are not perfectly balanced, so this addition takes this into account.

In [ ]:
model.fit(
        training_data, training_labels, epochs=EPOCHS,
        verbose=1, class_weight=class_weight, 
        batch_size=BATCH_SIZE,
        validation_data=(val_data, val_labels),
)